In [ ]:
sharedinputpath = snakemake.params["sharedinputpath"]
desired_regions = snakemake.params.aggregated_regions
weatherdata = snakemake.input.weatherdata

cf_file = snakemake.output.cf_file

In [ ]:
onshore_turbine = "Vestas_V112_3MW"
offshore_bottom_turbine = "oedb:V164"
offshore_floating_turbine = "oedb:V164"

panel = "CSi"
orientation = "latitude_optimal"

In [ ]:
geodata_files = {
    "onshore": snakemake.input.euroshape,
    "offshore_bottom": snakemake.input.eurooffshoreshape,
}

In [ ]:
import logging

import atlite

logging.basicConfig(level=logging.INFO)

import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np

In [ ]:
%psource atlite.Cutout.convert_and_aggregate

In [ ]:
boundaries = []
for geodata_file_name, geodata_file_path in geodata_files.items():
    print(geodata_file_path)
    boundaries.append(
        gpd.read_file(geodata_file_path)
    )

boundaries = pd.concat(boundaries).bounds

In [ ]:
boundaries = boundaries.groupby(lambda x: "bountry").agg(
    {"minx": "min", "miny": "min", "maxx": "max", "maxy": "max"}
)

# boundaries["minx"] = boundaries["minx"] - 2
# boundaries["miny"] = boundaries["miny"] - 2
# boundaries["maxx"] = boundaries["maxx"] + 2
# boundaries["maxy"] = boundaries["maxy"] + 2

In [ ]:
ds=xr.open_dataset(weatherdata,chunks="auto")

In [ ]:
cutout = atlite.Cutout(
    path="../3_intermediate_data/intermediatecutout.nc",
    data=ds.sel(
        x=slice(
            boundaries.loc["bountry", "minx"],
            boundaries.loc["bountry", "maxx"],
        ),
        y=slice(
            boundaries.loc["bountry", "miny"],
            boundaries.loc["bountry", "maxy"],
        ),
    ),
)

cutout.prepare()

In [ ]:
# Solar CF

In [ ]:
%psource atlite.Cutout.convert_and_aggregate

In [ ]:
cf_solar = cutout.pv(
    panel=panel,
    orientation=orientation,
    capacity_factor_timeseries=True,
).astype(np.float32)

In [ ]:
# Onshore wind CF

In [ ]:
cf_windon = cutout.wind(
    turbine=onshore_turbine, capacity_factor_timeseries=True
).astype(np.float32)

In [ ]:
# Offshore wind CF

In [ ]:
cf_windoff_bottom = cutout.wind(
    turbine=offshore_bottom_turbine, capacity_factor_timeseries=True
).astype(np.float32)

In [ ]:
cf = xr.concat(
    [cf_solar, cf_windon, cf_windoff_bottom],
    pd.Index(["Solar", "Windonshore", "Windoffshore"], name="Tech"),
)

In [ ]:
cf.to_netcdf(cf_file, 
             encoding={"capacity factor": {
                 'dtype': 'int16', 
                 'scale_factor': 0.001, 
                 '_FillValue': -99,
                 'zlib': True,
                 'complevel': 1,
            }})